# Домашнее задание №3

#### Текст домашнего задания

1. Загрузить файл длиной не менее 2000 символов.
2. Составить программу, которая считает число уникальных слов в тексте (без критерия схожести).
3. Составить программу, которая считает число гласных и согласных букв. 
4. Составить программу, которая считает число предложений, их длину (число слов) и число (количество) раз использования каждого слова в тексте с критерием схожести (критерий схожести слов выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние Левенштейна).
5. Вывести 10 наиболее часто встречаемых слов.

*p.s. Рекомендую перед решением задания проанализировать задачу и обосновать алгоритм ее решения в текстовом виде. В процессе написания кода использовать комментарии.*

#### Цель задания

Обучение работе с файлами. Анализ файла. Изучение критериев схожести.

#### Анализ задания


Основные этапы выполнения задания:
1. Проверка наличия файла с заданным параметром длинны и его загрузка.
2. Предобработка текста для дальнейшей работы (приведение текста к нижнему регистру, очистка текста от знаков пунктуации). Определение количества уникальных слов в отредактированном тексте.
3. Определение количества гласных и согласных букв.
4. Подсчёт предложений и их длины с помощью токенизации в spaCy. Подсчёт количества раз использования каждого слова с критерием схожести с помощью лемматизации и библиотеки стоп-слов из spaCy.  
В spacy есть функции для токенизации, лемматизации, библиотека стоп-слов.  
    - Токенизация - разбиение фразы, предложения, абзаца или всего текста на более мелкие единицы - токены.  
    - Лемматизация - поиск корней слов (лемм) в соответствие со словарём и преобразование слова в его базовую форму (если она есть в словаре).  
    - Стоп-слова - часто встречающиеся слова, которые не несут значимой информации.  
5. Вывод наиболее часто встречаемых слов в обработанном документе.

#### Решение домашнего задания

#### 1. Импорт необходимых для работы модулей. 

In [62]:
import string # констаны и функции для работы со строками
import spacy # библиотека для анализа текстов, предоставляет инструменты для работы с текстами ()
from collections import Counter # счётчик

#### 2. Объявение переменных для последующих тестов функций.

In [63]:
if __name__ == '__main__':
    link_empty_file = 'data/Empty.txt' # пустой файл
    link_none = 'data/None.txt' # файл отсутствует
    link_short_file = 'data/Short.txt' # короткий файл
    link_good_file = 'data/Henry_After_Twenty_Years.txt' # подходящий файл
    link_good_file_2 = 'data/Charlie_and_the_Chocolate_Factory.txt' # 2ой подходящий файл
    length = 2000
    links = link_short_file, link_empty_file, link_good_file, link_good_file_2, link_none
    # чтобы позже получить список ссылок на файлы, которые соответсвуют заданию

#### 3. Функция загрузки и проверки файла на соответствие условию домашнего задания.

In [64]:
def check_file(link_check: str, length_check: int):
    """
    Функция проверяет наличие файла в папке и его соответствие заданной длине.
    Закрывает за собой файл.
    Args:
        link_check (str): ссылка на файл.
        length_check (int): минимальное количество символов внутри файла для продолжения работы с ним.
    Returns tuple, состоящий из:
        str: строка, информирующая о результате проверки, при проваленной проверке.
        check (int): параметр успешности проверки, 0 - провалена, 1 - пройдена.
        content (str): при успешной проверке, возвращаем содержимое файла.  
    """
    try:
        with open(link_check, 'r', encoding='utf-8') as file_check: # открываем файл по ссылке
            file_text = file_check.read() # забираем содержимое файла
        file_check.close # закрываем файл
        symbols = len(file_text) # считаем количество символов
        if symbols == 0: # проверяем не пустой ли файл
            check = 0
            return f'Указанный файл {link_check} пуст. Выберите другой файл, длинной не менее {length_check} символов.', check
        elif symbols < length_check: # проверяем соответсвие количества символов заданному значению
            check = 0
            return f'Файл не соответствует заданию. Выберите другой файл, длинной не менее {length_check} символов.', check
        else:
            check = 1 # успешая проверка
            return file_text, check
    except FileNotFoundError: # ловим ошибку отсутствия файла в папке
        check = 0
        return f'При открытии файла произошла ошибка: указанного файла нет в директории.', check
    except Exception as unexpected_error:  # Ловим другие возможные ошибки
        check = 0
        return f'Произошла ошибка: {unexpected_error}', check

#### Тест функции check_file.

In [65]:
if __name__ == '__main__':
    checked_texts = []
    for i in links:
        print(check_file(i, length)[0][:100] + '\n') # отобразим первые 100 символов файла (если там файл), чтобы не грузить вывод
        if check_file(i, length)[1] == 1:
            checked_texts.append(check_file(i, length)[0]) # список, проверенных текстов

Файл не соответствует заданию. Выберите другой файл, длинной не менее 2000 символов.

Указанный файл data/Empty.txt пуст. Выберите другой файл, длинной не менее 2000 символов.

THE POLICEMAN ON the beat moved up the avenue impressively. The impressiveness was habitual and not 

There are five children in this book:
AUGUSTUS GLOOP A greedy boy
VERUCA SALT A girl who is spoiled 

При открытии файла произошла ошибка: указанного файла нет в директории.



#### 4. Функция предобработки текста.

In [66]:
def text_preprocessing(file_text: str):
    """
    Функция приводит все буквы в тексте к нижнему регистру.
    Удаляет все символы из перечня !"#$%&'()*+,-./:;<=>?@[]^_`{|}~0123456789“”‘’…«»—\
    Args:
        file_text (str): текст файла, для предобработки
    Returns:
        file_text (str): обработанный текст
    """
    file_text = file_text.lower() # приводим текст к нижнему регистру
    marks = string.punctuation + string.digits + '“”‘’…«»—' # собираем все символы, которые хотим убрать из текста
    for i in marks:
        file_text = file_text.replace(i, ' ') # проходим по тексту и заменяем символы пробелами
    return file_text

#### 4.* Была задумка усложнить, но слишком много случаев, которые попробуй ещё предусмотри

_text = text.replace("'ve", " have")_  
_text = text.replace("n't", " not")_  
_text = text.replace("'m", " am")_  
_text = text.replace("'ll", " will")_  
_text = text.replace("'re", " are")_  
_text = text.replace("d'you", "do you")_  
_text = text.replace("it's", "it is") #'s может быть ещё и has, для упрощения приведём к is_  
_text = text.replace("that's", "that is") #'s может быть ещё и has, для упрощения приведём к is_  
_text = text.replace("here's", "here is") #'s может быть ещё и has, для упрощения приведём к is_  
_text = text.replace("'d", " had") #'d может быть ещё и would, и should, для упрощения приведём к had_  

#### Тест функции text_preprocessing.

In [67]:
if __name__ == '__main__':
    preprocessing_texts = []
    for i in checked_texts:
        print(text_preprocessing(i)[:100] + ' ...\n')
        preprocessing_texts.append(text_preprocessing(i)) # список с предобработанными текстами

the policeman on the beat moved up the avenue impressively  the impressiveness was habitual and not  ...

there are five children in this book 
augustus gloop a greedy boy
veruca salt a girl who is spoiled  ...



#### 5. Функция опеделения и подсчёта уникальных слов.

In [68]:
def unique_words_in_text(file_text: str):
    """
    Функция определяет уникальные слова, с помощью множества и считает их.
    Args:
        file_text (str): текст, в котором необходимо подсчитать количество уникальных слов.
    Returns:
        int: число уникальных слов в тексте.
    """
    words = file_text.split() # получаем все слова в список, разделяя текст по пробелам
    unique_words = set(words) # с помощью особенности типа set хранить уникальные объекты, находим все уникальные слова в тексте
    return len(unique_words), unique_words # функией len получаем количество уникальных слов

#### Тест функции unique_words_in_text

In [69]:
if __name__ == '__main__':
    for i in preprocessing_texts:
        print(unique_words_in_text(i)[0])
        print(unique_words_in_text(i)[1])

314
{'next', 'club', 'lost', 'though', 'pile', 'track', 'wits', 'partner', 'sounds', 'll', 'few', 'but', 'him', 'stand', 'took', 'rain', 'made', 'a', 'policeman', 'agreed', 'kind', 'keen', 'our', 'early', 'going', 'long', 'all', 'good', 'pretty', 'swagger', 'done', 'cigar', 'appointment', 'match', 'ago', 'bet', 'half', 'like', 'lit', 'doesn', 'twirled', 'five', 'waiting', 'around', 'rather', 'impressiveness', 'slowed', 'raised', 'in', 'might', 'belonged', 'lights', 'until', 'staunchest', 'have', 'conditions', 'night', 'is', 'and', 'adown', 'to', 'edge', 'reassuringly', 'funny', 'since', 'habitual', 'at', 'places', 'hope', 'unlighted', 'if', 'groove', 'takes', 'he', 'me', 'business', 'show', 'then', 'doors', 'said', 'officer', 'right', 'little', 'been', 'pacific', 'left', 'impressively', 'light', 'stands', 'not', 'block', 'twirling', 'wells', 'years', 'large', 'after', 'new', 'has', 'announced', 'spectators', 'slight', 'down', 'near', 'brady', 'worked', 'guardian', 'step', 'just', 'peac

#### 6. Функция поиска количества гласных и согласных букв в тексте.

In [70]:
def letters_counter(file_text: str):
    """
    Функция поиска гласных и согласных букв английского алфавита в заданном тексте.
    Args:
        file_text (str): текст, в котором осуществляется поиск
    Returns:
        count_consonant (int): количетсво согласных букв
        count_vowels (int): количетсво гласных букв
    """
    vowels = 'aeiou' # гласные
    consonants = 'bcdfghjklmnpqrstvwxz' # согласные
    count_vowels = 0 # счётчик для гласных
    count_consonants = 0 # счётчик для согласных
    for i in file_text:
        if i in vowels:
            count_vowels += 1
        elif i in consonants:
            count_consonants += 1
    return count_consonants, count_vowels

#### Тест функции letters_counter.

In [71]:
if __name__ == '__main__':
    for i in preprocessing_texts:
        print(f'В тексте {letters_counter(i)[0]} согласных и {letters_counter(i)[1]} гласных')

В тексте 1637 согласных и 1015 гласных
В тексте 29459 согласных и 18323 гласных


#### 7. Функция для подсчёта количества предложений и их длины (т.е. количества слов в предложении).

In [72]:
def count_text_items(file_text: str):
    nlp = spacy.load("en_core_web_sm") # загрузка модели spacy для английского языка
    text = nlp(file_text) # nlp() - модель, которая возвращает текст
    num_sentences = len(list(text.sents)) # считаем число предложений в тексте
    sentences = {}
    for i in list(text.sents):
        sentence_text = str(i).strip() # убираем лишние пробелы и получаем текст предложения
        if sentence_text:  # проверяем, что предложение не пустое  
            word_count = len(sentence_text.split()) # считаем количество слов в предложении
            sentences[sentence_text] = word_count
    return num_sentences, sentences

#### Тест функции count_text_items.

In [73]:
if __name__ == '__main__':
    print(f'В заданном тексте {count_text_items(checked_texts[0])[0]} предложния/ий.')
    for j in count_text_items(checked_texts[0])[1]:
        print(f'Длина предложения {j} составляет {count_text_items(checked_texts[0])[1].get(j)} слов')

В заданном тексте 46 предложния/ий.
Длина предложения THE POLICEMAN ON the beat moved up the avenue impressively. составляет 10 слов
Длина предложения The impressiveness was habitual and not for show, for spectators were few. составляет 12 слов
Длина предложения The time was barely 10 o'clock at night, but chilly gusts of wind with a taste of rain in them had well nigh depeopled the streets. составляет 26 слов
Длина предложения Trying doors as he went, twirling his club with many intricate and artful movements, turning now and then to cast his watchful eye adown the pacific thoroughfare, the officer, with his stalwart form and slight swagger, made a fine picture of a guardian of the peace. составляет 46 слов
Длина предложения The vicinity was one that kept early hours. составляет 8 слов
Длина предложения Now and then you might see the lights of a cigar store or of an all-night lunch counter; but the majority of the doors belonged to business places that had long since been closed. сост

#### 8. Функция для подсчёта количества раз использования каждого слова с критерием схожести с помощью лемматизации и библиотеки стоп-слов из spaCy.

In [74]:
def count_words(file_text: str):
    nlp = spacy.load("en_core_web_sm") # загрузка модели spacy для английского языка
    text = nlp(file_text.lower()) # модель возвращает текст, в нижнем регистре
    processed_nlp_text = [] # список слов после лемматизации и удаления стоп-слов
    for token in text:
        if token.is_punct or token.is_space or token.is_stop or token.is_digit: # пропускаем пунктуацию, пробелы, стоп-слова и цифры
            continue
        processed_nlp_text.append(token.lemma_) # добавляем лемму
    statistic = Counter(processed_nlp_text)
    return statistic

#### Тест функции count_words.

In [75]:
if __name__ == '__main__':
    nlp_words = (count_words(checked_texts[0])) # словарь слов с учётом критерия схожести
    for key, value in nlp_words.items():
        print(f'Слово: {key} - количество раз в тексте: {value}')

Слово: policeman - количество раз в тексте: 7
Слово: beat - количество раз в тексте: 1
Слово: move - количество раз в тексте: 1
Слово: avenue - количество раз в тексте: 1
Слово: impressively - количество раз в тексте: 1
Слово: impressiveness - количество раз в тексте: 1
Слово: habitual - количество раз в тексте: 1
Слово: spectator - количество раз в тексте: 1
Слово: time - количество раз в тексте: 4
Слово: barely - количество раз в тексте: 1
Слово: o'clock - количество раз в тексте: 2
Слово: night - количество раз в тексте: 3
Слово: chilly - количество раз в тексте: 1
Слово: gust - количество раз в тексте: 1
Слово: wind - количество раз в тексте: 1
Слово: taste - количество раз в тексте: 1
Слово: rain - количество раз в тексте: 1
Слово: nigh - количество раз в тексте: 1
Слово: depeople - количество раз в тексте: 1
Слово: street - количество раз в тексте: 1
Слово: try - количество раз в тексте: 1
Слово: door - количество раз в тексте: 4
Слово: go - количество раз в тексте: 3
Слово: twir

#### 9. Функция наиболее часто встречающихся слов.

In [ ]:
def rate_words(dict_of_words: dict, number: int):
     return sorted(dict_of_words.items(), key=lambda item: item[1])[-number:] # сортируем по возрастанию и выводим number последних слов

In [83]:
if __name__ == '__main__':
    n = 10
    print(f'{n} наиболее часто встречаемых слов в тексте:')
    for word, cnt in rate_words(nlp_words, n):
        print(f'Слово {word} встречается в тексте {cnt} раз')

10 наиболее часто встречаемых слов в тексте:
Слово time встречается в тексте 4 раз
Слово door встречается в тексте 4 раз
Слово ago встречается в тексте 4 раз
Слово west встречается в тексте 4 раз
Слово pretty встречается в тексте 4 раз
Слово say встречается в тексте 5 раз
Слово jimmy встречается в тексте 5 раз
Слово man встречается в тексте 6 раз
Слово year встречается в тексте 6 раз
Слово policeman встречается в тексте 7 раз
